# Task 1

In [1]:
!pip install  datasets evaluate jiwer librosa
!pip install --upgrade bitsandbytes transformers==4.50.0 accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 22.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine

## Imoprt package

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import sys
sys.path.append('/content/drive/MyDrive/aicup')
import json
import torch
import random
import librosa
import zipfile
import evaluate
import numpy as np
import pandas as pd
from datasets import Dataset, Audio, load_dataset, Features, Value
from aicup import (DataCollatorSpeechSeq2SeqWithPadding,
      transcribe_with_timestamps,
      collate_batch_with_prompt_template,
      generate_annotated_audio_transcribe_parallel,OpenDeidBatchSampler)
from transformers import (
    WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor,
    WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer
)

## Download audio file

In [5]:
zip_path = "/content/Training_Dataset_Phase1_Sample.zip"
extract_dir = "/content/audio_dataset"
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
  zip_ref.extractall(extract_dir)

audio_files = []
for root, dirs, files in os.walk(extract_dir):
  for file in files:
    if file.lower().endswith('.wav'):
      audio_files.append(os.path.join(root, file))

print(f"found {len(audio_files)} audio files")
# for path in audio_files:
#   print(path)

found 210 audio files


In [6]:
def set_torch_seed(seed=0):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benckmark = False
  if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_torch_seed()

In [7]:
t1_train_audio_folder = "/content/audio_dataset/sample/Training_Dataset"
t1_train_transcription_file = "/content/audio_dataset/sample/task1_answer.txt"
transcripts, dataset_list = {}, []

with open(t1_train_transcription_file, "r", encoding="utf-8") as f:
  for line in f:
    if line.strip():
      parts = line.strip().split("\t", 1)
      if len(parts) == 2:
        filename, transcript = parts
        transcripts[filename] = transcript

for file in sorted(os.listdir(t1_train_audio_folder)):
  if file.endswith(".wav") and file.split(".")[0] in transcripts:
    try:
      file_path = os.path.join(t1_train_audio_folder, file)
      audio_array, sr = librosa.load(file_path, sr=16000)
      dataset_list.append({"audio":
                 {
                  "path":file_path,
                  "array":audio_array,
                  "sampling_rate":sr
                 },
                 "sentence": transcripts[file.split(".")[0]]})
    except Exception as e:
      print(e)
      print(f"Can't read {file_path}:{e}")

dataset = Dataset.from_pandas(pd.DataFrame(dataset_list))

In [ ]:
dataset[0]['audio']['array'][:20]

[4.803150659427047e-05,
 0.0002169541548937559,
 0.0003004224272444844,
 0.00028273859061300755,
 0.00016738526755943894,
 3.0343711841851473e-05,
 -5.1913782954216003e-05,
 -5.303439684212208e-05,
 -2.441985998302698e-06,
 6.212922744452953e-05,
 0.00010938383638858795,
 5.031493492424488e-05,
 -0.00010732980445027351,
 -0.0003319760144222528,
 -0.0005285177612677217,
 -0.0006197181064635515,
 -0.000516027444973588,
 -0.0002812005113810301,
 -1.3065175153315067e-05,
 0.0002058700192719698]

In [ ]:
print("audio sample:",len(dataset[0]['audio']['array']))
print("audio duration:",len(dataset[0]['audio']['array'])/16000)

audio sample: 385808
audio duration: 24.113


In [8]:
split_ratio = 0.8
train_size = int(len(dataset) * split_ratio)
dataset = dataset.train_test_split(train_size=train_size,
     test_size=len(dataset) - train_size, shuffle=True, seed=42)
train_dataset = dataset["train"]
test_dataset = dataset["test"]

print(f"Train: {len(train_dataset)} samples, Test: {len(test_dataset)} samples")

Train: 40 samples, Test: 10 samples


In [ ]:
test_dataset[0]

{'audio': {'array': [1.2341886758804321e-05,
   4.606170114129782e-05,
   4.8660323955118656e-05,
   6.228292477317154e-05,
   6.102980114519596e-05,
   5.93119184486568e-05,
   4.395644646137953e-05,
   -1.577078364789486e-06,
   -2.6318011805415154e-06,
   -1.7798389308154583e-05,
   -6.965897046029568e-05,
   -0.00010042288340628147,
   -0.00010713399387896061,
   -0.00011050817556679249,
   -8.20929417386651e-05,
   -2.5230285245925188e-05,
   -4.519271897152066e-06,
   6.629881681874394e-06,
   3.2405368983745575e-05,
   6.682961247861385e-05,
   6.309454329311848e-05,
   8.025206625461578e-05,
   9.123224299401045e-05,
   4.70051309093833e-05,
   3.643246600404382e-05,
   -1.1599069694057107e-05,
   3.08093149214983e-06,
   -9.25939530134201e-05,
   -0.00013775890693068504,
   -0.00011558737605810165,
   -0.00016050878912210464,
   -0.00012973748380318284,
   -0.00011225536582060158,
   -4.354957491159439e-05,
   1.6912177670747042e-05,
   -4.020167398266494e-05,
   -5.3548719733

## Model import

In [9]:
model_name = "openai/whisper-tiny"  #"small", "medium", "large"
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name)
tokenizer = WhisperTokenizer.from_pretrained(model_name)
tokenizer.padding_side = 'left'
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name)
model.generation_config.language = None
model.generation_config.task = "transcribe"
model.generation_config.forced_decoder_ids = None

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

### Pre-evaluate

In [10]:
from jiwer import mer
from transformers.models.whisper.english_normalizer import BasicTextNormalizer


def calculate_mer(ground_truth_texts, predicted_texts):
  """ Mix Error Rate (MER) English only"""
  mer_scores = {}
  total_mer = 0
  count = 0

  normalizer = BasicTextNormalizer()

  for filename, ref_text in ground_truth_texts.items():
    if filename in predicted_texts:
      pred_text = predicted_texts[filename]
      ref_text = normalizer(ref_text)
      pred_text = normalizer(pred_text)
      mer_score = mer(ref_text, pred_text)
      mer_scores[filename] = mer_score
      total_mer += mer_score
    else:
      mer_scores[filename] = 1
      total_mer += 1
    count += 1

  average_mer = total_mer / count if count != 0 else 0
  return mer_scores, average_mer

def evaluate_mer(model, processor, dataset):
  predictions = {}
  references = {}

  for sample in dataset:
    filename = sample["audio"]["path"].split("/")[-1].split(".")[0]
    audio_array = sample['audio']['array']
    sr = sample['audio']['sampling_rate']
    input_features = processor.feature_extractor(
        audio_array,
        sampling_rate=sr,
        return_tensors="pt"
    ).input_features.to(model.device)
    with torch.no_grad():
      predicted_ids = model.generate(input_features)
      transcription = processor.tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)[0].strip()

    predictions[filename] = transcription
    references[filename] = sample["sentence"].strip()

  mer_scores, avg_mer = calculate_mer(references, predictions)
  return mer_scores, avg_mer, predictions, references

In [11]:
mer_scores, avg_mer, predictions, references = evaluate_mer(model, processor, test_dataset)
print("All mer scores:\n",mer_scores)
print("Average mer scores:\n",avg_mer)
print("Predict :\n",predictions)
print("Answer :\n",references)

You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, 50259], [2, 50359], [3, 50363]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


All mer scores:
 {'10194': 0.039473684210526314, '1639': 0.09565217391304348, '1482': 0.02197802197802198, '2372': 0.05714285714285714, '1933': 0.08333333333333333, '1645': 0.05813953488372093, '23': 0.0, '2127': 0.024390243902439025, '218': 0.14285714285714285, '1461': 0.14285714285714285}
Average mer scores:
 0.06658241350782279
Predict :
 {'10194': "As you're comfortable doing that, you can scan your body again noticing any places where it might feel different than it felt before. No need to judge the feelings. No need to judge if it's the same or different. Just noticing it. And now just let yourself return to normal breath. Inhales and exhales at whatever pace feels natural to you. And then let yourself return to the room like over your focus.", '1639': "I know you're having a lot of panic and it's really uncomfortable, but I don't know how adding another issue on top of that is going to make you feel better. Well, it's not like anything I'm currently doing is really effectively m

## Dataset

In [ ]:
# print(processor.feature_extractor(test_dataset[2]['audio']['array'],
#     sampling_rate=test_dataset[2]['audio']["sampling_rate"])['input_features'])
print(tokenizer(test_dataset[2]['sentence']).input_ids)

[50258, 50363, 6271, 11, 291, 362, 732, 2493, 13, 286, 362, 732, 2493, 13, 663, 311, 2074, 13, 286, 600, 2726, 6928, 766, 293, 286, 519, 300, 390, 257, 665, 818, 26, 457, 1310, 286, 603, 312, 10879, 783, 300, 3537, 264, 917, 295, 6897, 13, 10517, 1296, 586, 293, 550, 11, 1953, 466, 437, 309, 307, 300, 534, 2203, 281, 483, 1096, 293, 3287, 10585, 5493, 294, 257, 25150, 636, 309, 1177, 380, 652, 2020, 281, 2844, 493, 1203, 586, 13, 821, 815, 312, 721, 291, 643, 11, 457, 1953, 466, 437, 2203, 281, 483, 1096, 293, 437, 393, 483, 1096, 13, 50257]


In [12]:
def preprocess_dataset(batch):

  audio = batch["audio"]
  batch["input_features"] = processor.feature_extractor(audio["array"],
    sampling_rate=audio["sampling_rate"]).input_features[0]
  batch["labels"] = tokenizer(batch["sentence"]).input_ids

  return batch

train_dataset = train_dataset.map(preprocess_dataset, remove_columns=dataset.column_names["train"]) #,num_proc=4
test_dataset = test_dataset.map(preprocess_dataset, remove_columns=dataset.column_names["test"])

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [13]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [ ]:
d = [test_dataset[i] for i in range(2)]
batch = data_collator(d)
print(batch['input_features'])
print(batch['labels'])

tensor([[[-0.8390, -0.9967, -0.9967,  ..., -0.9967, -0.9967, -0.9967],
         [-0.9967, -0.9967, -0.9967,  ..., -0.9967, -0.9967, -0.9967],
         [-0.9850, -0.9967, -0.9967,  ..., -0.9967, -0.9967, -0.9967],
         ...,
         [-0.9967, -0.9967, -0.9967,  ..., -0.9967, -0.9967, -0.9967],
         [-0.9967, -0.9967, -0.9967,  ..., -0.9967, -0.9967, -0.9967],
         [-0.9967, -0.9967, -0.9967,  ..., -0.9967, -0.9967, -0.9967]],

        [[-0.8342, -0.8342, -0.8342,  ..., -0.8342, -0.8342, -0.8342],
         [-0.8342, -0.8342, -0.8342,  ..., -0.8342, -0.8342, -0.8342],
         [-0.8342, -0.8342, -0.8342,  ..., -0.8342, -0.8342, -0.8342],
         ...,
         [-0.8342, -0.8342, -0.8342,  ..., -0.8342, -0.8342, -0.8342],
         [-0.8342, -0.8342, -0.8342,  ..., -0.8342, -0.8342, -0.8342],
         [-0.8342, -0.8342, -0.8342,  ..., -0.8342, -0.8342, -0.8342]]])
tensor([[50363, 10884,   291,   434,  4619,   884,   300,   291,   393, 11049,
           428,  1772,   797, 21814, 

In [14]:
from jiwer import mer
from transformers.models.whisper.english_normalizer import BasicTextNormalizer


normalizer = BasicTextNormalizer()

def mixed_tokenizer(text):
  text = normalizer(text.strip())
  tokens = []
  temp_token = ""
  # print(text)
  for char in text:
    if '\u4e00' <= char <= '\u9fff':
      if temp_token:
          tokens.append(temp_token)
          temp_token = ""
      tokens.append(char)
    elif char.isspace():
      if temp_token:
        tokens.append(temp_token)
        temp_token = ""
    else:
      temp_token += char
  if temp_token:
    tokens.append(temp_token)
  return tokens

def compute_metrics(eval_pred):
  predictions, labels = eval_pred

  decoded_preds = processor.batch_decode(predictions, skip_special_tokens=True)
  decoded_labels = processor.batch_decode(labels, skip_special_tokens=True)

  decoded_preds = [normalizer(pred.strip()) for pred in decoded_preds]
  decoded_labels = [normalizer(label.strip()) for label in decoded_labels]

  paired = [
      (ref, hyp) for ref, hyp in zip(decoded_labels, decoded_preds)
      if ref.strip() != "" and hyp.strip() != ""
  ]

  filtered_labels, filtered_preds = zip(*paired) if paired else ([], [])
  if len(filtered_labels) == 0:
    return {"mer": 1.0}

  ref_tokens = [mixed_tokenizer(t) for t in filtered_labels]
  pred_tokens = [mixed_tokenizer(t) for t in filtered_preds]
  # print(ref_tokens)
  # print(pred_tokens)
  ref_strs = [" ".join(tokens) for tokens in ref_tokens]
  pred_strs = [" ".join(tokens) for tokens in pred_tokens]

  try:
    score = mer(ref_strs, pred_strs)
  except Exception as e:
    print("Error during MER computation:", e)
    score = 1.0

  return {"mer": score}

## Finetune decoder only (option)

In [15]:
for param in model.model.encoder.parameters():
  param.requires_grad = False

In [16]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"Trainable parameters: {trainable_params:,}")
print(f"Total parameters:     {total_params:,}")
print(f"Trainable ratio:      {trainable_params / total_params:.2%}")

Trainable parameters: 29,552,256
Total parameters:     37,760,640
Trainable ratio:      78.26%


## Training & args

In [ ]:
train_dataset

In [17]:
training_args = Seq2SeqTrainingArguments(
  output_dir="/content/whisper-tiny",
  report_to="none",
  num_train_epochs=5,
  per_device_train_batch_size=4,
  per_device_eval_batch_size=4,
  evaluation_strategy="epoch",
  save_strategy="epoch",
  predict_with_generate=True,
  logging_dir="/content/whisper-tiny/logs",
  logging_steps=1,
  fp16=True,
  learning_rate=1e-5,
  warmup_ratio=0.1,
  gradient_accumulation_steps=4,
  dataloader_num_workers=4,
)

trainer = Seq2SeqTrainer(
  model=model,
  args=training_args,
  train_dataset=train_dataset,
  eval_dataset=test_dataset,
  tokenizer=processor.tokenizer,
  data_collator=data_collator,
  compute_metrics=compute_metrics
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-17-a78f010729cc>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [18]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Mer
1,0.625300,1.053699,0.064198
2,0.496500,0.972735,0.066502
3,1.024100,0.938691,0.063197


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358, 50359, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=10, training_loss=0.9392309159040451, metrics={'train_runtime': 31.5794, 'train_samples_per_second': 6.333, 'train_steps_per_second': 0.317, 'total_flos': 3348168376320000.0, 'train_loss': 0.9392309159040451, 'epoch': 3.4})

In [19]:
save_directory = "/content/whisper-tiny"

model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)
processor.save_pretrained(save_directory)

print(f"Model saved to {save_directory}")

Model saved to /content/whisper-tiny


## Output result

In [20]:
valid_dataset_list = []
t1_vaild_audio_folder = "/content/audio_dataset/sample/Validation_Dataset"
for file in sorted(os.listdir(t1_vaild_audio_folder)):
  if file.endswith(".wav"):
    try:
      file_path = os.path.join(t1_vaild_audio_folder, file)
      audio_array, sr = librosa.load(file_path, sr=16000)
      valid_dataset_list.append({"audio": {"path":file_path,'array':audio_array,'sampling_rate':sr},
                                 "sentence": ""})
    except Exception as e:
      print(e)
      print(f"Can't read {file_path}")

valid_dataset = Dataset.from_pandas(pd.DataFrame(valid_dataset_list))

In [ ]:
valid_dataset

Dataset({
    features: ['audio', 'sentence'],
    num_rows: 150
})

In [21]:
# model_name = "/content/drive/MyDrive/aicup/models/whisper-tiny"
# model = WhisperForConditionalGeneration.from_pretrained(model_name)
# processor = WhisperProcessor.from_pretrained(model_name)
model.generation_config.language = 'en'
model.to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

output_file = "/content/audio_dataset/sample/submission/task1_answer.txt"
json_output_file = "/content/audio_dataset/sample/task1_answer_timestamps.json"
_mapping = {}

with open(output_file, "w", encoding="utf-8") as f:
  for _file in valid_dataset:
    result = transcribe_with_timestamps(_file,model,processor)
    _mapping[_file['audio']['path'].split("/")[-1].split(".")[0]] = result
    f.write(f"{_file['audio']['path'].split('/')[-1].split('.')[0]}\t{result['text']}\n")

with open(json_output_file, "w", encoding="utf-8") as f:
  json.dump(_mapping, f, ensure_ascii=False)

# Task 2

In [22]:
task2_train_answer = r"/content/audio_dataset/sample/task2_answer.txt"
task2_train_transcribe = r"/content/audio_dataset/sample/task1_answer.txt"
task2_train_data = '/content/audio_dataset/sample/task2_train.tsv'
generate_annotated_audio_transcribe_parallel(task2_train_answer, task2_train_transcribe, task2_train_data, num_processes=4)

process annotation file...
annotation file done
processing each medical file
1346	I pull up the index and sure enough, the guy who had the property died with a will, and all I need to confirm is this one person was the sole heir of the will or the person in the will to get the property. But it's in storage, the file is in storage. They're starting to put everything in storage now which fucks me because these are — typically the late 60s and 70s are when I need probates and those older ones, they order on Fridays.	DATE:60s\nDATE:70s\nDATE:Fridays

70
All medical file done
write out to tsv format...
tsv format dataset done


## Read dataset

In [23]:
from datasets import load_dataset, Features, Value

task2_data = load_dataset("csv",data_files=task2_train_data, delimiter='\t',
  features = Features({'fid': Value('string'),'content': Value('string'),'label':Value('string')}),
  column_names=['fid','content','label'])

Generating train split: 0 examples [00:00, ? examples/s]

In [24]:
task2_data

DatasetDict({
    train: Dataset({
        features: ['fid', 'content', 'label'],
        num_rows: 70
    })
})

In [25]:
from collections import Counter


ctr = Counter()

for i in task2_data['train']:
  phi_labelwvalue = i['label'].split("\\n")
  phi_label = [j.split(":")[0] for j in phi_labelwvalue]
  ctr.update(phi_label)

print(ctr)

Counter({'DATE': 46, 'DURATION': 32, 'PERSONALNAME': 15, 'TIME': 11, 'PHI': 11, 'DOCTOR': 7, 'FAMILYNAME': 5, 'SET': 3, 'PATIENT': 3, 'COUNTRY': 2, 'ORGANIZATION': 1, 'LOCATION-OTHER': 1, 'DISTRICT': 1, 'AGE': 1})


## Model imoprt

In [26]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig
import torch
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model


task2_model_name = "deepseek-ai/deepseek-llm-7b-base"
bos = '<|endoftext|>'
eos = '<|END|>'
pad = '<|pad|>'
sep = '\n\n####\n\n'

special_tokens_dict = {
    'eos_token': eos,
    'bos_token': bos,
    'pad_token': pad,
    'sep_token': sep
}

tokenizer = AutoTokenizer.from_pretrained(task2_model_name, use_fast=False, trust_remote_code=True)
tokenizer.padding_side = 'left'
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)
config = AutoConfig.from_pretrained(
    task2_model_name,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    sep_token_id=tokenizer.sep_token_id,
    output_hidden_states=False
)

model = AutoModelForCausalLM.from_pretrained(
    task2_model_name,
    quantization_config=bnb_config,
    config=config,
    trust_remote_code=True
)
model.resize_token_embeddings(len(tokenizer))
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj","k_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

print(f"{tokenizer.pad_token}: {tokenizer.pad_token_id}")

tokenizer_config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/584 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


pytorch_model.bin.index.json:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.6k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

<|pad|>: 100017


In [27]:
def collate_batch_with_prompt_template(batch, tokenizer, template =
  "<|endoftext|> __CONTENT__\n\n####\n\n__LABEL__ <|END|>", IGNORED_PAD_IDX = -100):

  texts = [template.replace("__LABEL__", data['label']).replace("__CONTENT__",
    data['content']) for data in list(batch)]
  encoded_seq = tokenizer(texts, padding=True)

  indexed_tks = torch.tensor(encoded_seq['input_ids'])
  attention_mask = torch.tensor(encoded_seq['attention_mask'])
  encoded_label = torch.tensor(encoded_seq['input_ids'])
  encoded_label[encoded_label == tokenizer.pad_token_id] = IGNORED_PAD_IDX

  return indexed_tks, encoded_label, attention_mask

In [29]:
from torch.utils.data import Dataset, DataLoader


train_data = list(task2_data['train'])
train_dataloader = DataLoader(train_data, batch_size=2, shuffle=False, collate_fn=lambda batch: collate_batch_with_prompt_template(batch, tokenizer))
titer = iter(train_dataloader)
tks, labels, masks= next(titer)
print(tks.shape)
print(tks)
print(labels)
print(masks)
print(tokenizer.decode(tks[0]))
print(tokenizer.decode(tks[1]))
next(iter(titer))


torch.Size([2, 172])
tensor([[100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017,
         100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017,
         100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017,
         100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017,
         100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017, 100017,
         100000, 100016,  32259,     11,    304,  10743,    359,    744,    207,
           1972,  11547,     11,    614,   8450,     13,   2086,    359,      6,
             82,   8548,    276,    830,   6163,     11,   1329,     30,  85298,
             13,   2086,    395,      6,    647,   1019,    565,    304,      6,
             76,  25009,  15593,     13,    304,   3655,    304,      6,     76,
          25009,   3571,     13,    304,      6,    313,   3370,    803,   3507,
            276,   2449,     13,   1273,    304,    536,    254,  18612,   3571,
       

(tensor([[100000, 100016,  32259,     11,    304,    608,   3370,     11,  30351,
              11,  30351,     13,  13290,     11,    558,    953,    558,    344,
             304,    487,    313,   2149,    254,   3693,   6044,     11,    437,
             521,   4732,    786,    207,     21,   4223,   4084,     11,   1323,
             344,    661,    340,    661,   3270,    837,    437,     30,   2366,
              11,    441,    430,    521,     11,    304,   2059,     11,    304,
            1378,    395,    661,    245,    761,   7709,    430,    830,     11,
             304,    830,    279,    245,   6185,   3370,     11,    285,    937,
             395,    661,    245,    761,   4541,   1317,   1957,    279,    285,
             558,    359,   5322,   1079,   1266,    691,  42019,    327,    521,
             254,   5724,     11,    441,    953,    525,     11,    570,    285,
             304,    481,   3734,    280,   1019,    304,    438,   1090,    245,
            2603

In [ ]:
results = tokenizer(
    [f"{bos} Yeah, I imagine it would — sorry, go ahead. So it's supposed to work immediately, right? Yep. So we'll see if I'm productive tomorrow. I hope I'm productive today. I've actually been trying to plan. If I do the titles today, then I can do my laundry tomorrow. Right. I probably could bring my computer and do titles while I'm doing my laundry. If I was — but I won't do that.{sep}DATE:tomorrow\nDATE:today\nDate:today {eos}",
    f"{bos} I imagine it{sep}PHI:Null {eos}"]
    ,padding=True
)
print(results['attention_mask'][0])
print(results['attention_mask'][1])
print(tokenizer.decode(results['input_ids'][0]))
print(tokenizer.decode(results['input_ids'][1]))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
<|endoftext|><|endoftext|> Yeah, I imagine it would — sorry, go ahead. So it's supposed to work immediately, right? Yep. So we'll see if I'm productive tomorrow. I hope I'm productive today. I've actually been trying to plan. If I do the titles today, then I can do my laundry tomorrow. Right. I probably could bring my computer and do ti

In [30]:
BATCH_SIZE = 8
bucket_train_dataloader = DataLoader(train_data,
  batch_sampler=OpenDeidBatchSampler(train_data, BATCH_SIZE),
  collate_fn=lambda batch: collate_batch_with_prompt_template(batch, tokenizer),
  pin_memory=True)

In [31]:
model.print_trainable_parameters()

trainable params: 7,864,320 || all params: 6,898,724,864 || trainable%: 0.1140


## Training

In [32]:
from transformers import get_scheduler
from tqdm import tqdm
import torch
import os


EPOCHS = 5
GRADIENT_ACCUMULATION_STEPS = 4
LEARNING_RATE = 5e-4
MAX_GRAD_NORM = 1.0
EARLY_STOP_PATIENCE = 6
output_dir = "/content/qlora-deepseek"
os.makedirs(output_dir, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
total_steps = len(bucket_train_dataloader) * EPOCHS // GRADIENT_ACCUMULATION_STEPS
lr_scheduler = get_scheduler(
    "cosine",
    optimizer=optimizer,
    num_warmup_steps=200,
    num_training_steps=total_steps,
)
best_loss = float("inf")
early_stop_counter = 0

for epoch in range(EPOCHS):
    total_loss = 0.0
    model.train()

    progress_bar = tqdm(enumerate(bucket_train_dataloader), total=len(bucket_train_dataloader), desc=f"Epoch {epoch+1}")

    for step, (seqs, labels, masks) in progress_bar:
        input_ids = seqs.to(device)
        labels = labels.to(device)
        attention_mask = masks.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss / GRADIENT_ACCUMULATION_STEPS
        loss.backward()
        total_loss += loss.item()

        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0 or (step + 1) == len(bucket_train_dataloader):
            torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)
            optimizer.step()
            lr_scheduler.step()
            model.zero_grad()

        progress_bar.set_postfix(
            loss=loss.item() * GRADIENT_ACCUMULATION_STEPS,
            lr=lr_scheduler.get_last_lr()[0]
        )

    avg_loss = total_loss / len(bucket_train_dataloader)
    print(f"\nEpoch {epoch+1} average loss: {avg_loss:.4f}")

    # ==== save best model ====
    if avg_loss < best_loss:
        best_loss = avg_loss
        early_stop_counter = 0
        best_path = os.path.join(output_dir, "best_adapter")
        model.save_pretrained(best_path)
        tokenizer.save_pretrained(best_path)
        print(f"Best model saved at {best_path} with loss {avg_loss:.4f}")
    else:
        early_stop_counter += 1
        print(f"No improvement. Early stop patience: {early_stop_counter}/{EARLY_STOP_PATIENCE}")

    # ==== Early Stopping ====
    if early_stop_counter >= EARLY_STOP_PATIENCE:
        print(f"Early stopping triggered at epoch {epoch+1}. Best loss: {best_loss:.4f}")
        break


model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Epoch 1:   0%|          | 0/9 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
Epoch 1:   0%|          | 0/9 [00:13<?, ?it/s]


KeyboardInterrupt: 

## Load finetune model (Optional)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,AutoConfig
from peft import PeftModel

bos = '<|endoftext|>'
eos = '<|END|>'
pad = '<|pad|>'
sep = '\n\n####\n\n'
base_model_name = "deepseek-ai/qlora-deepseek-llm-7b-base"
adapter_path = "/content/qlora-deepseek"

tokenizer = AutoTokenizer.from_pretrained(adapter_path, trust_remote_code=True)
tokenizer.padding_side = 'left'
config = AutoConfig.from_pretrained(
    base_model_name,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    sep_token_id=tokenizer.sep_token_id,
    output_hidden_states=False
)

bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_use_double_quant=True)
base_model = AutoModelForCausalLM.from_pretrained(base_model_name,config=config, quantization_config=bnb_config, trust_remote_code=True)
base_model.resize_token_embeddings(len(tokenizer))

model = PeftModel.from_pretrained(base_model, adapter_path)
model.eval().cuda()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/584 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


pytorch_model.bin.index.json:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.6k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(100019, 4096, padding_idx=100017)
        (layers): ModuleList(
          (0-29): 30 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
         

## Output result

In [36]:
task2_valid_data = '/content/audio_dataset/sample/submission/task1_answer.txt'
valid_data = load_dataset("csv", data_files=task2_valid_data, delimiter='\t',
  features = Features({'fid': Value('string'),'content': Value('string')}),
  column_names=['fid', 'content'])
valid_list = list(valid_data['train'])
valid_data

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['fid', 'content'],
        num_rows: 150
    })
})

In [33]:
with open(r'/content/audio_dataset/sample/task1_answer_timestamps.json', 'r', encoding='utf-8') as file:
  audio_timestamps = json.load(file)

In [34]:
from collections import defaultdict


train_phi_category = ['PATIENT', 'DOCTOR', 'USERNAME','FAMILYNAME','PROFESSION',
             'ROOM', 'DEPARTMENT', 'HOSPITAL', 'ORGANIZATION', 'STREET', 'CITY',
             'DISTRICT','COUNTY','STATE', 'COUNTRY', 'ZIP', 'LOCATION-OTHER',
             'AGE',
             'DATE', 'TIME', 'DURATION', 'SET',
             'PHONE', 'FAX', 'EMAIL', 'URL', 'IPADDR',
             'SSN', 'MEDICALRECORD', 'HEALTHPLAN', 'ACCOUNT',
             'LICENSE', 'VEHICLE', 'DEVICE', 'BIOID', 'IDNUM',
             'OTHER']

def get_anno_format(infos, audio_timestamps):
  anno_list = []
  phi_dict = defaultdict(list)
  for line in infos.split("\n"):
    if ":" not in line:
      continue
    key, value = line.split(":", 1)
    key = key.strip()
    value = value.strip()
    if key in train_phi_category and value:
      phi_dict[key].append(value)

  remaining_timestamps = audio_timestamps.copy()
  used_indices = set()

  for phi_key, phi_values in phi_dict.items():
    for phi_value in phi_values:
      phi_tokens = phi_value.lower().strip().split()

      for i in range(len(remaining_timestamps) - len(phi_tokens) + 1):
        if any((i + j) in used_indices for j in range(len(phi_tokens))):
          continue

        match = True
        for j, phi_token in enumerate(phi_tokens):
          tsd_word = remaining_timestamps[i + j]['word'].replace("Ġ", "").replace("▁", "").strip().lower()
          if tsd_word != phi_token:
            match = False
            break

        if match:
          anno_list.append({
              "phi": phi_key,
              "st_time": remaining_timestamps[i]['start'],
              "ed_time": remaining_timestamps[i + len(phi_tokens) - 1]['end'],
              "entity": phi_value
          })
          for j in range(len(phi_tokens)):
            used_indices.add(i + j)
          break
  return anno_list

def aicup_predict(model, tokenizer, _input, audio_timestamps, template = "<|endoftext|> __CONTENT__\n\n####\n\n"):
  seeds = [template.replace("__CONTENT__", data['content']) for data in _input]
  sep = tokenizer.sep_token
  eos = tokenizer.eos_token
  pad = tokenizer.pad_token
  model.eval()
  device = model.device
  texts = tokenizer(seeds, return_tensors = 'pt', padding=True).to(device)
  outputs = []

  with torch.no_grad():
    output_tokens = model.generate(**texts, max_new_tokens=32, do_sample=False,
    temperature=None,
    top_p=None,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id)
    preds = tokenizer.batch_decode(output_tokens, skip_special_tokens=False)
    for idx , pred in enumerate(preds):
      if "Null" in pred:
        continue
      phi_infos = pred[pred.index(sep)+len(sep):].replace(pad, "").replace(eos, "").strip()
      annotations = get_anno_format(phi_infos,audio_timestamps[_input[idx]['fid']]['segments'])
      for annotation in annotations:
        outputs.append(f'{_input[idx]["fid"]}\t{annotation["phi"]}\t{annotation["st_time"]}\t{annotation["ed_time"]}\t{annotation["entity"]}')
  return outputs

In [37]:
import torch
from tqdm import tqdm

BATCH_SIZE = 10
with open("/content/audio_dataset/sample/submission/task2_answer.txt",'w',encoding='utf8') as f:
  for i in tqdm(range(0, len(valid_list), BATCH_SIZE)):
    with torch.no_grad():
      data = valid_list[i:i+BATCH_SIZE]
      outputs = aicup_predict(model, tokenizer, data, audio_timestamps)
      for o in outputs:
        f.write(o)
        f.write('\n')

100%|██████████| 15/15 [05:16<00:00, 21.12s/it]
